In [7]:
#import all libraries

from __future__ import print_function, division
from bs4 import BeautifulSoup #importing soup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
#importing libraries
import re
import requests
import time
import random
import pandas as pd
requests.__path__

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [8]:
df = pd.read_csv('SCRAPERLINKDATA.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1085 entries, 0 to 1084
Data columns (total 3 columns):
ID              1085 non-null int64
Link            1085 non-null object
Neighborhood    1085 non-null object
dtypes: int64(1), object(2)
memory usage: 25.5+ KB


In [10]:
df.head()

,ID,Link,Neighborhood
0,25841637,https://www.airbnb.com/rooms/25841637?location...,downtown
1,24392338,https://www.airbnb.com/rooms/24392338?location...,downtown
2,25838923,https://www.airbnb.com/rooms/25838923?location...,downtown
3,13971471,https://www.airbnb.com/rooms/13971471?location...,downtown
4,34929016,https://www.airbnb.com/rooms/34929016?location...,downtown


In [15]:
driver = webdriver.Chrome(chromedriver)

In [16]:
counter = 1
full_data = []
listing_data =[]

for link in df['Link']:
    driver.get(link)
    time.sleep(8+2*random.random()) #Not to get dinged by Airbnb
    
    print (counter) #counter to keep track of #of links
    
    ######Initialize Variables########

    price = 0

    #Listing Info
    listing_type =''
    num_guests = 0
    num_bedroom = 0
    num_beds = 0
    num_bath = 0
    shared_bath = ""
    num_ammenities = 0

    #Review Data
    number_review = 0
    overall_review = 0
    accuracy = 0
    communication = 0
    cleanliness = 0
    location = 0
    check_in = 0
    value = 0
    

    
    ##WAIT UNTIL PAGE LOADS and CHECK IF PAGE HAS COMPONENTS OF A LISTING PAGE###
    
    #class="_1rbmiub1" #informational links at the end of the page
    #another class that occurs at bottom of page the facebook link
    
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "_1rbmiub1")))
    except:
        try:
            print ("Loading is Taking Longer Than Expected")
        
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "_fy9czq")))
        except:
            print("Listing Delisted")
            
            #populate row with 0 values
            listing_data = [price,listing_type,num_guests,num_bed,num_bath,shared_bath,num_ammenities,number_review,overall_review,accuracy,communication,cleanliness,value]
            full_data.append(listing_data)    
            
            continue
    
    #########################
    
    
    ######WEBSCRAPING#######
    
    
    # New heading
    soup = BeautifulSoup(driver.page_source, 'html.parser')
          
    #Find type of listing  Entire, Private, Shared, Hotel, listings get delisted, need to account for this.
    listing = soup.find(class_ = "_1p3joamp").text

    #may need an if statement to find out which kind of listing it is
    listing_type = listing.split()[0]
    
    
    #list of all prices and fees obtain final price

    #sometimes dates no longer available
    
    try:
        prices = []

        for price in soup.find_all(class_ = "_j1kt73"):
            prices.append(price.text)

        final_price = int(re.sub('[^0-9]','', prices[-1]))
        price = final_price #final price is listed at the end
    except:
        
        print('Dates Not Available')
        price = None
        pass
     


    #Order is not always the same, need to consolidate and write a better RegEx    
    #Description Data
    
    description =[]
    for link in soup.find_all(class_ = "_czm8crp"):
        description.append(link.text)
    
    description_string = ' '.join(description)

    #Number of Guests

    #Check if Information is present

    y = description_string
    x = '([0-9.]+) guest*'

    if re.search(x,y):

        num_guests = float(re.search(x,y).group().split()[0])


    
    #Number of Beds
    
    y = description_string
    x = '([0-9.]+) bed*'

    #Check if Information is present
    if re.search(x,y):
        num_bed = float(re.search(x,y).group().split()[0])

    
    #Number of Bathrooms, some people don't list number of baths
    try:
        bath_lists = description_string.split('bath')
        # we can throw away the last part because all of it comes after
        # the last instance of 'bath'
        bath_lists = bath_lists[:-1]
        num_bath = 0

        nums = re.findall('\d+\.?\d*',bath_lists[0])
        # if there's numbers before 'bath' take the last one

        num_bath += float(nums[-1])
    # otherwise assume it's one bath
    except:
        num_bath += 1
    
    
    #Shared Bathroom?
    try:    
        y = description_string
        x = 'shared'
    
        shared_bath = re.search(x,y).group()
    except:
        shared_bath = 'notshared'
        pass
    
    #Amenities Data
    
    try:
        ammenities =[]
        for link in soup.find_all(class_="_4xosax"):
            ammenities.append(link.text)
        
        num_ammenities = len(ammenities)
    
    except:
        pass
    

    #Number of reviews, sometimes there are no reviews for a new listing, try except to cover this area
    
    try:
        #overall review score this gives an error if there are no reviews
        aggregate_rating = re.search('([0-5](.[0-9])?)', str(soup.find(itemprop = "ratingValue")))
        overall_review = float(aggregate_rating.group()) #group converts the results into a float
    
    except:               
        pass #iterate next loop and skip rest of the review extraction

    #Number of Reviews

    if soup.find(class_ = "_ff6jfq"):
        number_review = int(soup.find(class_ = "_ff6jfq").text)
    
        #Accuracy
    
        x = '"Accuracy","localized_rating":"([0-9.]+)"'

        m=re.search(x,soup.text)

        accuracy = float(m.group(1))
    
        #Communication
        x = '"Communication","localized_rating":"([0-9.]+)"'

        m=re.search(x,soup.text)

        communication = float(m.group(1))
    
        #Cleanliness
        x = '"Cleanliness","localized_rating":"([0-9.]+)"'

        m=re.search(x,soup.text)

        cleanliness = float(m.group(1))
    
        #Location
        x = '"Location","localized_rating":"([0-9.]+)"'

        m=re.search(x,soup.text)
    
        location = float(m.group(1))
    
        #Check-In
        x = '"Check-in","localized_rating":"([0-9.]+)"'

        m=re.search(x,soup.text)

        check_in = float(m.group(1))    
   
        #Value
        x = '"Check-in","localized_rating":"([0-9.]+)"'

        m=re.search(x,soup.text)

        value = float(m.group(1))
    
    #Consolidated Data Line
    listing_data = [price,listing_type,num_guests,num_bed,num_bath,shared_bath,num_ammenities,number_review,overall_review,accuracy,communication,cleanliness,value,check_in,location]

    full_data.append(listing_data)    
    ###### WEBSCRAPING DONE#######
    print(listing_data)
    
    if counter % 50 == 0:
        print("Taking a Break")
        time.sleep(120) #Taks a 2 Min (120 second) break after 50 links 
    
    counter = counter +1

1
[158, 'Private', 4.0, 1.0, 1.0, 'shared', 5, 86, 4.5, 4.7, 4.7, 4.8, 4.8, 4.8, 4.6]
2
[225, 'Entire', 4.0, 2.0, 1.0, 'notshared', 4, 109, 4.5, 4.8, 4.9, 4.7, 4.8, 4.8, 4.9]
3
[130, 'Private', 2.0, 1.0, 1.0, 'shared', 5, 73, 4.5, 4.7, 4.7, 4.9, 4.9, 4.9, 4.6]
4
[79, 'Shared', 1.0, 1.0, 1, 'notshared', 5, 230, 5.0, 4.9, 5.0, 4.9, 4.9, 4.9, 5.0]
5
[86, 'Private', 1.0, 1.0, 1.0, 'shared', 4, 10, 5.0, 4.8, 5.0, 4.4, 4.9, 4.9, 5.0]
6
[253, 'Entire', 5.0, 2.0, 1.0, 'notshared', 4, 49, 5.0, 4.8, 5.0, 4.8, 5.0, 5.0, 5.0]
7
[146, 'Private', 2.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
8
[120, 'Private', 1.0, 1.0, 1.0, 'shared', 4, 169, 5.0, 4.8, 4.9, 4.7, 4.9, 4.9, 4.9]
9
[395, 'Entire', 6.0, 1.0, 1.5, 'notshared', 4, 185, 4.5, 4.8, 4.8, 4.7, 4.6, 4.6, 4.8]
10
[298, 'Entire', 2.0, 1.0, 1.0, 'notshared', 4, 9, 5.0, 5.0, 5.0, 5.0, 4.9, 4.9, 4.9]
11
[192, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 364, 5.0, 5.0, 4.9, 4.9, 4.9, 4.9, 4.8]
12
[334, 'Entire', 3.0, 2.0, 1.0, 'notshared', 4, 410,

96
[286, 'Entire', 3.0, 1.0, 1.0, 'notshared', 4, 205, 4.5, 4.6, 4.8, 4.4, 4.8, 4.8, 4.9]
97
[251, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 13, 5.0, 4.8, 5.0, 4.6, 5.0, 5.0, 5.0]
98
[139, 'Shared', 2.0, 1.0, 1.0, 'shared', 4, 21, 4.5, 5.0, 4.9, 4.0, 4.7, 4.7, 4.5]
99
[259, 'Room', 2.0, 1.0, 1.0, 'shared', 6, 0, 0, 0, 0, 0, 0, 0, 0]
100
[181, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 325, 4.5, 4.7, 4.9, 4.5, 4.8, 4.8, 4.8]
Taking a Break
101
[358, 'Entire', 2.0, 1.0, 1.0, 'notshared', 4, 153, 5.0, 4.8, 4.9, 4.9, 4.9, 4.9, 4.9]
102
[329, 'Entire', 2.0, 1.0, 1.5, 'notshared', 4, 230, 4.5, 4.6, 4.8, 4.4, 4.7, 4.7, 4.8]
103
[293, 'Room', 2.0, 1.0, 1.0, 'notshared', 4, 49, 4.5, 4.5, 4.5, 4.8, 4.6, 4.6, 4.9]
104
[474, 'Entire', 6.0, 2.0, 1.0, 'notshared', 6, 86, 4.5, 4.6, 4.7, 4.3, 4.7, 4.7, 4.8]
105
[326, 'Entire', 2.0, 1.0, 1.0, 'notshared', 5, 85, 5.0, 4.8, 4.9, 4.8, 5.0, 5.0, 5.0]
106
[299, 'Room', 2.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
107
[184, 'Private', 3.0, 1.0, 1.0, 'shar

191
[536, 'Entire', 2.0, 1.0, 1.0, 'notshared', 4, 15, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
192
[480, 'Room', 2.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
193
[164, 'Entire', 2.0, 1.0, 1.0, 'notshared', 5, 5, 5.0, 5.0, 4.8, 5.0, 4.5, 4.5, 5.0]
194
[560, 'Room', 2.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
195
[192, 'Private', 2.0, 1.0, 1.0, 'shared', 5, 212, 5.0, 4.9, 4.9, 4.9, 4.9, 4.9, 5.0]
196
[299, 'Room', 2.0, 2.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
197
[423, 'Entire', 3.0, 1.0, 1.0, 'notshared', 4, 3, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
198
[762, 'Entire', 5.0, 2.0, 1.0, 'notshared', 4, 112, 4.5, 4.8, 4.6, 4.5, 4.7, 4.7, 4.9]
199
[621, 'Entire', 5.0, 2.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
200
[582, 'Entire', 2.0, 1.0, 2.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
Taking a Break
201
[125, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
202
[336, 'Entire', 2.0, 1.0, 1.5, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
203
[316, 'Entire', 

289
[429, 'Entire', 7.0, 2.0, 1.0, 'notshared', 5, 0, 0, 0, 0, 0, 0, 0, 0]
290
[135, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
291
[370, 'Entire', 3.0, 1.0, 1.0, 'notshared', 4, 9, 5.0, 4.8, 5.0, 4.9, 4.9, 4.9, 5.0]
292
[201, 'Private', 3.0, 1.0, 1.0, 'notshared', 6, 78, 4.5, 4.8, 4.9, 4.5, 4.9, 4.9, 4.9]
293
[350, 'Private', 3.0, 1.0, 1.0, 'notshared', 5, 34, 5.0, 4.9, 5.0, 4.8, 5.0, 5.0, 5.0]
294
[841, 'Entire', 6.0, 1.0, 1.0, 'notshared', 4, 39, 5.0, 4.8, 5.0, 4.5, 4.8, 4.8, 4.9]
295
[186, 'Private', 4.0, 1.0, 1.0, 'shared', 4, 14, 5.0, 4.6, 5.0, 4.5, 4.9, 4.9, 5.0]
296
[270, 'Entire', 1.0, 1.0, 1.0, 'notshared', 4, 25, 5.0, 5.0, 5.0, 4.8, 5.0, 5.0, 5.0]
297
[112, 'Private', 1.0, 1.0, 1.0, 'shared', 6, 11, 4.5, 4.8, 5.0, 4.6, 5.0, 5.0, 5.0]
298
[435, 'Entire', 5.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
299
Loading is Taking Longer Than Expected
Listing Delisted
299
[452, 'Entire', 2.0, 1.0, 1.0, 'notshared', 4, 5, 4.0, 4.2, 4.2, 4.2, 4.2, 4.2, 4.0]
3

381
[316, 'Entire', 4.0, 2.0, 1.0, 'notshared', 4, 45, 4.5, 4.8, 5.0, 4.6, 5.0, 5.0, 4.9]
382
[309, 'Entire', 4.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
383
[90, 'Shared', 2.0, 1.0, 1.0, 'shared', 4, 71, 4.5, 4.7, 4.8, 4.6, 4.8, 4.8, 5.0]
384
[209, 'Entire', 4.0, 1.0, 1.0, 'notshared', 4, 45, 4.0, 4.5, 4.7, 4.1, 4.7, 4.7, 4.7]
385
[169, 'Private', 2.0, 1.0, 2.0, 'shared', 4, 69, 4.5, 4.9, 4.8, 4.3, 4.9, 4.9, 4.9]
386
[440, 'Room', 5.0, 1.0, 2.0, 'notshared', 5, 11, 4.5, 4.4, 4.4, 4.8, 4.5, 4.5, 4.9]
387
[316, 'Entire', 4.0, 1.0, 1.0, 'notshared', 4, 79, 4.5, 4.7, 4.7, 4.8, 4.8, 4.8, 4.9]
388
[287, 'Entire', 4.0, 2.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
389
[174, 'Private', 2.0, 1.0, 1.0, 'shared', 5, 64, 4.0, 4.2, 4.4, 4.2, 4.5, 4.5, 4.8]
390
[369, 'Room', 6.0, 1.0, 1.0, 'notshared', 4, 31, 4.0, 4.1, 4.6, 4.3, 4.6, 4.6, 4.8]
391
[213, 'Room', 2.0, 1.0, 1.0, 'notshared', 4, 7, 4.0, 4.0, 4.0, 4.1, 4.4, 4.4, 5.0]
392
[260, 'Room', 2.0, 1.0, 1.0, 'notshared', 4, 0, 0, 

...

667
[58, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 50, 5.0, 4.9, 4.9, 4.9, 4.8, 4.8, 4.8]
668
[248, 'Entire', 3.0, 2.0, 1.0, 'notshared', 4, 12, 5.0, 4.7, 5.0, 4.6, 5.0, 5.0, 4.6]
669
[131, 'Private', 4.0, 1.0, 1.0, 'shared', 4, 156, 5.0, 4.9, 4.9, 4.9, 4.9, 4.9, 4.6]
670
[180, 'Private', 2.0, 1.0, 0.0, 'shared', 4, 281, 4.5, 4.6, 4.7, 4.3, 4.8, 4.8, 4.8]
671
[280, 'Private', 4.0, 2.0, 1.0, 'notshared', 4, 14, 4.0, 4.1, 4.8, 4.3, 4.3, 4.3, 4.6]
672
[145, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 278, 4.5, 4.6, 4.8, 4.3, 4.8, 4.8, 4.9]
673
[102, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 94, 5.0, 4.9, 5.0, 4.8, 5.0, 5.0, 4.8]
674
[159, 'Entire', 3.0, 1.0, 1.0, 'notshared', 4, 125, 4.5, 4.8, 4.9, 4.4, 4.7, 4.7, 4.4]
675
[135, 'Private', 2.0, 1.0, 1.0, 'shared', 6, 14, 4.5, 4.9, 4.8, 4.6, 5.0, 5.0, 4.8]
676
[135, 'Private', 6.0, 1.0, 1.0, 'shared', 4, 27, 4.5, 4.8, 4.8, 4.4, 4.9, 4.9, 4.8]
677
[157, 'Private', 3.0, 1.0, 1.0, 'shared', 4, 110, 4.5, 4.7, 4.7, 4.4, 4.6, 4.6, 4.6]
678
[158, 'Entire', 2

760
[141, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 113, 5.0, 4.8, 4.9, 4.8, 4.9, 4.9, 4.7]
761
[91, 'Private', 2.0, 1.0, 1.5, 'shared', 4, 225, 4.5, 4.7, 4.9, 4.1, 4.9, 4.9, 4.5]
762
[1074, 'Entire', 12.0, 4.0, 2.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
763
[226, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 23, 5.0, 4.7, 4.9, 4.8, 4.7, 4.7, 4.7]
764
[116, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 48, 5.0, 5.0, 5.0, 4.8, 5.0, 5.0, 4.7]
765
[73, 'Shared', 1.0, 1.0, 1, 'notshared', 6, 0, 0, 0, 0, 0, 0, 0, 0]
766
[81, 'Shared', 1.0, 1.0, 1.0, 'shared', 4, 32, 4.5, 4.7, 4.5, 4.5, 4.8, 4.8, 4.6]
767
[198, 'Private', 2.0, 1.0, 1.0, 'shared', 5, 52, 5.0, 4.7, 4.9, 4.5, 4.8, 4.8, 4.9]
768
[102, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 8, 5.0, 4.8, 5.0, 4.6, 5.0, 5.0, 5.0]
769
[609, 'Entire', 9.0, 3.0, 2.0, 'notshared', 4, 90, 4.5, 4.5, 4.8, 4.7, 4.8, 4.8, 4.3]
770
[73, 'Shared', 1.0, 1.0, 1, 'shared', 6, 0, 0, 0, 0, 0, 0, 0, 0]
771
[150, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 272, 5.0, 4.8, 4.9, 4.7, 4.9,

851
[113, 'Private', 1.0, 1.0, 1.0, 'shared', 4, 3, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
852
[360, 'Private', 4.0, 1.0, 1.0, 'shared', 4, 75, 5.0, 4.9, 5.0, 4.9, 5.0, 5.0, 4.6]
853
[79, 'Private', 2.0, 2.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
854
[113, 'Private', 1.0, 1.0, 1.0, 'shared', 6, 0, 0, 0, 0, 0, 0, 0, 0]
855
[226, 'Private', 4.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
856
[81, 'Private', 2.0, 0.0, 1.0, 'shared', 6, 8, 5.0, 4.8, 4.9, 4.5, 4.8, 4.8, 4.5]
857
[280, 'Entire', 4.0, 2.0, 1.0, 'notshared', 4, 36, 4.5, 4.1, 4.8, 4.6, 4.5, 4.5, 4.1]
858
[113, 'Private', 1.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
859
[677, 'Entire', 6.0, 2.0, 2.0, 'notshared', 5, 0, 0, 0, 0, 0, 0, 0, 0]
860
Loading is Taking Longer Than Expected
Listing Delisted
860
[476, 'Entire', 14.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
861
[163, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 78, 5.0, 4.9, 4.9, 5.0, 4.9, 4.9, 5.0]
862
[124, 'Private', 2.0, 1.0, 1.0, 'shared', 5, 4, 5.0, 4.

947
[349, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
948
[220, 'Entire', 2.0, 1.0, 0.0, 'notshared', 4, 3, 4.0, 5.0, 4.3, 4.3, 5.0, 5.0, 5.0]
949
[399, 'Room', 4.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
950
[463, 'Entire', 3.0, 1.0, 1.0, 'notshared', 4, 188, 5.0, 4.8, 4.9, 4.8, 4.9, 4.9, 4.9]
Taking a Break
951
[384, 'Room', 3.0, 1.0, 1.0, 'notshared', 5, 0, 0, 0, 0, 0, 0, 0, 0]
952
[469, 'Room', 3.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
953
[145, 'Private', 2.0, 1.0, 1.0, 'shared', 6, 56, 4.0, 4.0, 4.3, 4.2, 4.4, 4.4, 4.7]
954
[297, 'Private', 2.0, 1.0, 1.0, 'shared', 4, 54, 5.0, 4.8, 5.0, 5.0, 4.9, 4.9, 5.0]
955
[256, 'Yurt', 5.0, 2.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
956
[423, 'Entire', 4.0, 1.0, 1.0, 'notshared', 4, 37, 5.0, 5.0, 5.0, 4.9, 5.0, 5.0, 4.9]
957
[469, 'Room', 4.0, 1.0, 1.0, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
958
[419, 'Room', 3.0, 1.0, 1.0, 'notshared', 5, 0, 0, 0, 0, 0, 0, 0, 0]
959
[1040, 'Entire', 4.0, 2.0, 

1046
[219, 'Shared', 2.0, 1.0, 1.5, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
1047
[100, 'Shared', 2.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
1048
[395, 'Shared', 1.0, 1.0, 1, 'notshared', 2, 0, 0, 0, 0, 0, 0, 0, 0]
1049
[135, 'Shared', 2.0, 1.0, 1, 'notshared', 4, 7, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.8]
1050
[126, 'Shared', 3.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
Taking a Break
1051
[143, 'Shared', 2.0, 1.0, 1.0, 'shared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
1052
[47, 'Shared', 1.0, 1.0, 1, 'shared', 4, 14, 4.5, 4.6, 4.7, 4.7, 4.7, 4.7, 4.7]
1053
[147, 'Shared', 3.0, 1.0, 1, 'notshared', 4, 29, 5.0, 4.8, 4.9, 5.0, 5.0, 5.0, 4.6]
1054
[71, 'Shared', 1.0, 1.0, 1.0, 'shared', 4, 52, 5.0, 5.0, 4.9, 4.9, 5.0, 5.0, 4.8]
1055
[231, 'Shared', 1.0, 1.0, 1, 'notshared', 4, 0, 0, 0, 0, 0, 0, 0, 0]
1056
[45, 'Shared', 1.0, 1.0, 1.0, 'shared', 4, 4, 4.5, 4.8, 4.5, 4.5, 5.0, 5.0, 4.5]
1057
[198, 'Shared', 1.0, 1.0, 1, 'notshared', 4, 6, 5.0, 4.8, 5.0, 4.6, 5.0, 5.0, 5.0]
1058
[113, 'Shared', 1

In [45]:
driver.close()

In [43]:
len(full_data)

530

In [44]:
cols = ['price','listing_type','num_guests','num_bed','num_bath','shared_bath','num_ammenities','number_review','overall_review','accuracy','communication','cleanliness','value', 'check_in', 'location']
df2 = pd.DataFrame.from_records(full_data, columns= cols)

df2.to_csv('airbnbdatafinalwithallfeatures.csv')

In [29]:
df2.info

<bound method DataFrame.info of      price listing_type  num_guests  num_bed  num_bath shared_bath  \
0      158      Private         4.0      1.0       1.0      shared   
1      225       Entire         4.0      2.0       1.0   notshared   
2      130      Private         2.0      1.0       1.0      shared   
3       79       Shared         1.0      1.0       1.0   notshared   
4       78      Private         1.0      1.0       1.0      shared   
5      260       Entire         5.0      2.0       1.0   notshared   
6      146      Private         2.0      1.0       1.0   notshared   
7      120      Private         1.0      1.0       1.0      shared   
8      395       Entire         6.0      1.0       1.5   notshared   
9      298       Entire         2.0      1.0       1.0   notshared   
10     192      Private         2.0      1.0       1.0      shared   
11     334       Entire         3.0      2.0       1.0   notshared   
12     334       Entire         6.0      3.0       1.0   n

In [30]:
for i in range(0,len(df['Link'])):
               print()

In [31]:
len(df['Link'])

1085